In [1]:
import pandas as pd
import numpy as np
import random
import gc
from os import listdir
from keras.models import Sequential, load_model
from keras.layers import Dense, Activation, Dropout, BatchNormalization, LSTM
from sklearn.preprocessing import MinMaxScaler
np.random.seed(0)

Using Theano backend.


# Note types
- 0: nothing
- 1: step
- 2: hold start
- 3: hold/roll end
- 4: roll start
- M: mine

# Classes
- 0: nothing
- 1: one note
- 2: two notes
- 3: three or four notes
- 4: hold start
- 5: roll start
- 6: mine

In [11]:
samples_back_included_indices = [0, 1, 2, 3, 4, 6, 8, 9, 12, 16, 24, 36, 48]
samples_back_included = len(samples_back_included_indices)
num_classes = 7
num_features = 44
num_features_total = (num_features * samples_back_included)
save_files = listdir('data')

def get_features_for_index(beat_features, index):
    return beat_features[index] if index >= 0 else [0] * num_features

def get_class_for_index_expanded(notes, index):
    if index < 0:
        return [1, 0, 0, 0, 0, 0, 0]
    row = notes[index][0]
    (steps, holds, rolls, mines) = [row.count(char) for char in ['1', '2', '4', 'M']]
    if steps == 0 and mines == 0 and holds == 0 and rolls == 0:
        return [1, 0, 0, 0, 0, 0, 0]
    steps += (holds + rolls)
    return [int(i) for i in [False, steps == 1, steps == 2, steps > 2, holds > 0, rolls > 0, mines > 0]]

def get_class_for_index(notes, index):
    classes_expanded = get_class_for_index_expanded(notes, index)
    return [i for i in range(7) if classes_expanded[i]]
    
importance_rankings = [48, 24, 12, 16, 6, 8, 3, 4, 2, 1]
def get_beat_importance(index):
    for i in range(len(importance_rankings)):
        if index % importance_rankings[i] == 0:
            return i

def get_features_for_song(X, y, key):
    if '{0}_beat_features.csv'.format(key) in save_files and '{0}_notes.csv'.format(key) in save_files:
        beat_features_rotated = pd.read_csv('data/{0}_beat_features.csv'.format(key)).values
        notes = pd.read_csv('data/{0}_notes.csv'.format(key), converters={'0': lambda x: str(x)}).values
        beat_features = np.flipud(np.rot90(np.array(beat_features_rotated)))
        num_notes = min(len(notes), len(beat_features))
        new_beat_features = []
        for beat_feature_row, i in zip(beat_features, range(len(beat_features))):
            new_beat_feature_row = np.concatenate((beat_feature_row, [i % 48, get_beat_importance(i), i / 48, num_notes - i / 48]), axis=0)
            new_beat_features.append(new_beat_feature_row)

        for i in range(num_notes):
            class_num = get_class_for_index_expanded(notes, i)
            features = [feature for j in samples_back_included_indices for feature in get_features_for_index(new_beat_features, i - j)]
            X.append(features)
            y.append(class_num)

def build_training_data(songs):
    X = []
    y = []
    for song_data in songs:
        get_features_for_song(X, y, song_data[0])
    return np.array(X), np.array(y)

In [ ]:
# Total 243 songs
songs_to_use = pd.read_csv('data/songs_to_use.csv').values
np.random.shuffle(songs_to_use)

In [3]:
%%time
X_train, y_train = build_training_data(songs_to_use[:194])
X_test, y_test = build_training_data(songs_to_use[194:])

scaler = MinMaxScaler(feature_range=(0, 1))
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

X_train = np.reshape(X_train, (X_train.shape[0], samples_back_included, num_features))
X_test = np.reshape(X_test, (X_test.shape[0], samples_back_included, num_features))
gc.collect()

CPU times: user 1min 27s, sys: 22.8 s, total: 1min 49s
Wall time: 1min 59s


In [4]:
%%time
model = Sequential()

model.add(LSTM(128, batch_input_shape=[64, samples_back_included, num_features], stateful=True))
model.add(BatchNormalization())
model.add(Activation('softsign'))
model.add(Dropout(0.2))

model.add(Dense(num_classes))
model.add(BatchNormalization())
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adagrad',
              metrics=['accuracy'])

batch_size = 64
train_cutoff = int(len(X_train) / batch_size) * batch_size
test_cutoff = int(len(X_test) / batch_size) * batch_size
model.fit(X_train[:train_cutoff], y_train[:train_cutoff], nb_epoch=8, batch_size=batch_size, verbose=1, validation_data=(X_test[:test_cutoff], y_test[:test_cutoff]))
print (model.evaluate(X_test[:test_cutoff], y_test[:test_cutoff], batch_size=batch_size))

Train on 491968 samples, validate on 109248 samples
Epoch 1/8
491968/491968 [==============================] - 196s - loss: 0.8154 - acc: 0.8568 - val_loss: 0.5901 - val_acc: 0.8813
Epoch 2/8
491968/491968 [==============================] - 202s - loss: 0.5148 - acc: 0.8789 - val_loss: 0.4884 - val_acc: 0.8826
Epoch 3/8
491968/491968 [==============================] - 203s - loss: 0.4552 - acc: 0.8801 - val_loss: 0.4581 - val_acc: 0.8828
Epoch 4/8
491968/491968 [==============================] - 206s - loss: 0.4297 - acc: 0.8806 - val_loss: 0.4355 - val_acc: 0.8845
Epoch 5/8
491968/491968 [==============================] - 211s - loss: 0.4157 - acc: 0.8814 - val_loss: 0.4253 - val_acc: 0.8848
Epoch 6/8
491968/491968 [==============================] - 215s - loss: 0.4061 - acc: 0.8818 - val_loss: 0.4206 - val_acc: 0.8851
Epoch 7/8
491968/491968 [==============================] - 219s - loss: 0.3997 - acc: 0.8821 - val_loss: 0.4200 - val_acc: 0.8849
Epoch 8/8
109248/109248 [=============

# Full Model

In [15]:
%%time
X, y = build_training_data(songs_to_use)

scaler = MinMaxScaler(feature_range=(0, 1))
X = scaler.fit_transform(X)

X = np.reshape(X, (X.shape[0], samples_back_included, num_features))
gc.collect()

model = Sequential()

model.add(LSTM(128, batch_input_shape=[64, samples_back_included, num_features], stateful=True))
model.add(BatchNormalization())
model.add(Activation('softsign'))
model.add(Dropout(0.2))

model.add(Dense(num_classes))
model.add(BatchNormalization())
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adagrad',
              metrics=['accuracy'])

cutoff = int(len(X) / batch_size) * batch_size
model.fit(X[:cutoff], y[:cutoff], nb_epoch=8, batch_size=batch_size, verbose=1)
model.save('models/song_class_model.h5')

Epoch 1/8
601216/601216 [==============================] - 343s - loss: 0.7671 - acc: 0.8603   
Epoch 2/8
601216/601216 [==============================] - 345s - loss: 0.4905 - acc: 0.8798   
Epoch 3/8
601216/601216 [==============================] - 345s - loss: 0.4410 - acc: 0.8807   
Epoch 4/8
601216/601216 [==============================] - 347s - loss: 0.4200 - acc: 0.8819   
Epoch 5/8
601216/601216 [==============================] - 346s - loss: 0.4080 - acc: 0.8827   
Epoch 6/8
601216/601216 [==============================] - 347s - loss: 0.4002 - acc: 0.8832   
Epoch 7/8
601216/601216 [==============================] - 347s - loss: 0.3941 - acc: 0.8841   
Epoch 8/8
601216/601216 [==============================] - 349s - loss: 0.3895 - acc: 0.8849   
CPU times: user 1h 29min 30s, sys: 4min 38s, total: 1h 34min 8s
Wall time: 52min 21s
